# 1. Environment Setup
First, we need to load our API keys from the `.env` file.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the Gemini API key is loaded correctly
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file")
print("Gemini API Key loaded successfully.")

Gemini API Key loaded successfully.


In [2]:
# 2. Import LangChain and Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

# Import our custom prompts (now that the project is installed)
from prompts.scrip_writer_agent import SCRIPT_WRITER_AGENT_PROMPT


In [3]:
# 3. Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=gemini_api_key,
    temperature=0.7
)

print("Gemini LLM initialized successfully!")


Gemini LLM initialized successfully!


# 4. Dynamic Prompt with Pydantic Models

Now let's test the new dynamic prompt system that automatically updates when we modify the Pydantic models in `src/models/models.py`.


In [4]:
# Import the new dynamic prompt system
from prompts.scrip_writer_agent import SCRIPT_WRITER_AGENT_TEMPLATE, VIDEO_SCRIPT_PARSER, create_dynamic_prompt
from models.models import SceneType, ImageStyle, VoiceTone, TransitionType, HookTechnique

print("✅ Dynamic prompt system imported successfully!")
print(f"📋 Available Scene Types: {[t.value for t in SceneType]}")
print(f"🎨 Available Image Styles: {[s.value for s in ImageStyle]}")
print(f"🎤 Available Voice Tones: {[t.value for t in VoiceTone]}")
print(f"🔄 Available Transitions: {[t.value for t in TransitionType]}")
print(f"🎯 Available Hook Techniques: {[t.value for t in HookTechnique]}")


✅ Dynamic prompt system imported successfully!
📋 Available Scene Types: ['explanation', 'visual_demo', 'comparison', 'story_telling', 'hook', 'conclusion']
🎨 Available Image Styles: ['single_character', 'character_with_background', 'infographic', 'diagram_explanation', 'before_after_comparison', 'step_by_step_visual', 'four_cut_cartoon', 'comic_panel', 'speech_bubble', 'cinematic', 'close_up_reaction', 'wide_establishing_shot', 'split_screen', 'overlay_graphics', 'cutaway_illustration']
🎤 Available Voice Tones: ['excited', 'curious', 'serious', 'friendly', 'sad', 'mysterious', 'surprised', 'confident', 'worried', 'playful', 'dramatic', 'calm', 'enthusiastic']
🔄 Available Transitions: ['fade', 'slide_left', 'slide_right', 'zoom_in', 'zoom_out', 'dissolve', 'wipe', 'push', 'spin', 'flip', 'none']
🎯 Available Hook Techniques: ['shocking_fact', 'intriguing_question', 'visual_surprise', 'contradiction', 'mystery_setup']


In [5]:
# Create the dynamic prompt chain
chain = SCRIPT_WRITER_AGENT_TEMPLATE | llm | VIDEO_SCRIPT_PARSER

print("✅ Dynamic prompt chain created successfully!")
print(f"📝 Template type: {type(SCRIPT_WRITER_AGENT_TEMPLATE)}")
print(f"📋 Parser type: {type(VIDEO_SCRIPT_PARSER)}")
print(f"🔗 Chain type: {type(chain)}")

# Show format instructions length
format_instructions = VIDEO_SCRIPT_PARSER.get_format_instructions()
print(f"📋 Format instructions length: {len(format_instructions)} characters")


✅ Dynamic prompt chain created successfully!
📝 Template type: <class 'langchain_core.prompts.prompt.PromptTemplate'>
📋 Parser type: <class 'langchain_core.output_parsers.pydantic.PydanticOutputParser'>
🔗 Chain type: <class 'langchain_core.runnables.base.RunnableSequence'>
📋 Format instructions length: 5552 characters


# 5. Test Dynamic Prompt Generation

Let's test the dynamic prompt with a simple subject and save the results to the `/temp` folder.


In [ ]:
import json
import os
from datetime import datetime

def save_llm_result_to_temp(result, subject):
    """
    Save LLM result to notebooks/temp folder as JSON
    """
    # Create temp directory
    temp_dir = "/temp"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Generate filename: script_subject_datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    clean_subject = "".join(c for c in subject if c.isalnum() or c in (' ', '-', '_')).rstrip()
    clean_subject = clean_subject.replace(' ', '_')[:50]
    
    filename = f"script_{clean_subject}_{timestamp}.json"
    filepath = os.path.join(temp_dir, filename)
    
    # Save result as-is (convert to dict if Pydantic model)
    if hasattr(result, 'model_dump'):
        data = result.model_dump()
    else:
        data = result
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Saved to: {filepath}")
    return filepath

print("✅ Simple save function created!")


✅ Simple save function created!


In [7]:
# Test 1: Simple subject with dynamic prompt
test_subject = "Why do cats purr?"
test_language = "English"
test_max_scenes = 4

print(f"🧪 Testing dynamic prompt with subject: '{test_subject}'")
print(f"📝 Language: {test_language}")
print(f"🎬 Max scenes: {test_max_scenes}")
print("⏳ Generating...")

try:
    # Generate using dynamic prompt chain
    result = chain.invoke({
        "subject": test_subject,
        "language": test_language,
        "max_video_scenes": test_max_scenes
    })
    
    print("✅ SUCCESS! Dynamic prompt generation completed!")
    print(f"📄 Title: {result.title}")
    print(f"📄 Total scenes: {result.total_scene_count}")
    print(f"📄 Hook technique: {result.hook_scene.hook_technique.value}")
    print(f"📄 Hook scene type: {result.hook_scene.scene_type.value}")
    print(f"📄 Hook voice tone: {result.hook_scene.voice_tone.value}")
    
    # Save to temp folder
    saved_file = save_llm_result_to_temp(result, test_subject)
    
    print(f"\n🎯 Dynamic prompt benefits demonstrated:")
    print(f"  ✅ All enum values automatically included from Pydantic models")
    print(f"  ✅ Type-safe output parsing with PydanticOutputParser")
    print(f"  ✅ Result saved to temp folder as JSON")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


🧪 Testing dynamic prompt with subject: 'Why do cats purr?'
📝 Language: English
🎬 Max scenes: 4
⏳ Generating...
✅ SUCCESS! Dynamic prompt generation completed!
📄 Title: The Purrfect Mystery: Why Do Cats Purr?
📄 Total scenes: 6
📄 Hook technique: mystery_setup
📄 Hook scene type: hook
📄 Hook voice tone: mysterious
✅ Saved to: notebooks/temp/script_Why_do_cats_purr_20250914_235353.json

🎯 Dynamic prompt benefits demonstrated:
  ✅ All enum values automatically included from Pydantic models
  ✅ Type-safe output parsing with PydanticOutputParser
  ✅ Result saved to temp folder as JSON


# 6. Test Dynamic Updates

Let's demonstrate how the prompt automatically updates when we modify the Pydantic models. We'll add a new voice tone and see it reflected in the prompt.


In [ ]:
# Test dynamic updates by recreating the prompt
print("🔄 Testing dynamic prompt updates...")

# Recreate the dynamic prompt to see current enum values
new_prompt, new_parser = create_dynamic_prompt()

# Check if the prompt contains all current enum values
current_voice_tones = [t.value for t in VoiceTone]
print(f"📋 Current Voice Tones in models: {current_voice_tones}")

# Check if all voice tones are in the prompt
missing_tones = []
for tone in current_voice_tones:
    if tone not in new_prompt:
        missing_tones.append(tone)

if missing_tones:
    print(f"❌ Missing voice tones in prompt: {missing_tones}")
else:
    print("✅ All voice tones are present in the dynamic prompt!")

# Show a sample of the prompt to verify enum values are included
prompt_sample = new_prompt[new_prompt.find("**Available Voice Tones:**"):new_prompt.find("**IMPORTANT**: Use the EXACT lowercase values") + 100]
print(f"\n📝 Prompt sample showing voice tones:")
print(prompt_sample)

print(f"\n🎯 Dynamic update verification:")
print(f"  ✅ Prompt automatically includes all current enum values")
print(f"  ✅ No manual prompt updates needed when models change")
print(f"  ✅ LangChain PydanticOutputParser handles all parsing")


# 7. Custom Test Function

Create a reusable function to test different subjects and save results automatically.


In [ ]:
def test_dynamic_prompt(subject, language="English", max_scenes=5):
    """
    Test the dynamic prompt system with a given subject and save results
    
    Args:
        subject: The topic for the video script
        language: Language for the script
        max_scenes: Maximum number of scenes
    
    Returns:
        The generated VideoScript result
    """
    print(f"🧪 Testing dynamic prompt with subject: '{subject}'")
    print(f"📝 Language: {language}, Max scenes: {max_scenes}")
    print("⏳ Generating...")
    
    try:
        # Generate using dynamic prompt chain
        result = chain.invoke({
            "subject": subject,
            "language": language,
            "max_video_scenes": max_scenes
        })
        
        print("✅ SUCCESS! Generation completed!")
        print(f"📄 Title: {result.title}")
        print(f"📄 Total scenes: {result.total_scene_count}")
        print(f"📄 Hook technique: {result.hook_scene.hook_technique.value}")
        print(f"📄 Hook voice tone: {result.hook_scene.voice_tone.value}")
        
        # Save to temp folder
        saved_file = save_llm_result_to_temp(result, subject)
        print(f"💾 Saved to: {saved_file}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Simple test function created!")
print("Usage: test_dynamic_prompt('Your subject here')")


In [ ]:
# Test the custom function with different subjects
test_subjects = [
    "How does photosynthesis work?",
    "Why is the sky blue?",
    "How do birds fly?"
]

print("🧪 Testing multiple subjects with custom function...")

results = []
for i, subject in enumerate(test_subjects, 1):
    print(f"\n--- Test {i}: {subject} ---")
    result = test_dynamic_prompt(subject, max_scenes=3)
    if result:
        results.append(result)
    print()

print(f"✅ Completed {len(results)} successful tests!")
print(f"📁 All results saved to notebooks/temp/ folder")
print(f"🎯 Each test used dynamic enum values from Pydantic models")


# 8. Summary and Next Steps

## What We've Accomplished:

✅ **Dynamic Prompt System**: Created a system that automatically updates when Pydantic models change  
✅ **PydanticOutputParser Integration**: Type-safe output parsing with LangChain  
✅ **Automatic File Saving**: Results saved to `/temp` folder in multiple formats  
✅ **Reusable Test Functions**: Easy testing with different subjects  
✅ **Enum Value Validation**: All enum values automatically included from models  

## Key Benefits:

1. **🔄 Automatic Updates**: No manual prompt maintenance needed
2. **🛡️ Type Safety**: PydanticOutputParser ensures correct output format
3. **📁 Organized Storage**: Results automatically saved with timestamps
4. **🧪 Easy Testing**: Simple functions for testing different subjects
5. **📋 Complete Integration**: All LangChain features utilized

## Next Steps:

- Modify `src/models/models.py` to add new enum values
- Test that prompts automatically update
- Use the `test_dynamic_prompt()` function for new subjects
- Check saved files in `notebooks/temp/` folder


In [6]:
# 4. Create the advanced video script prompt template
video_script_prompt = PromptTemplate(
    input_variables=["subject", "language", "max_video_scenes"],
    template=SCRIPT_WRITER_AGENT_PROMPT
)

print("Advanced video script prompt template created successfully!")


Advanced video script prompt template created successfully!


In [11]:
# 5. Create video script generation function
def generate_video_script(subject, language="English", max_video_scenes=3):
    """
    Generate a complete video script using the advanced prompt template
    """
    # Format the prompt with variables
    formatted_prompt = video_script_prompt.format(
        subject=subject,
        language=language,
        max_video_scenes=max_video_scenes
    )
    
    # Create a human message
    message = HumanMessage(content=formatted_prompt)
    
    # Get response from LLM
    response = llm.invoke([message])

    return response.content


In [12]:
# Test the function with a simple topic
print("Testing video script generation...")
result = generate_video_script("How Elon must became a CEO of Tesla", "English", 5)
print(f"\nGenerated Video Script:\n{result}")

Testing video script generation...

Generated Video Script:
```json
{
  "VideoScript": {
    "title": "The Ocean's Salty Secret: Why is the Sea So Salty?",
    "main_character_description": "A friendly, curious marine biologist named Dr. Coral, with bright blue eyes and a warm smile. She wears a stylish lab coat over a casual shirt.",
    "overall_style": "Educational with engaging cartoon elements",
    "scenes": [
      {
        "scene_type": "HOOK",
        "hook_type": "SHOCKING_FACT",
        "visual_style": "INFOGRAPHIC",
        "needs_animation": false,
        "dialogue": "Did you know that humans indirectly consume about 1kg of sea salt every year through food?  But where does all that salt come from?",
        "voice_tone": "EXCITED",
        "transition": null
      },
      {
        "scene_type": "SETUP",
        "visual_style": "WIDE_ESTABLISHING_SHOT",
        "needs_animation": false,
        "dialogue": "We all know the ocean is salty, but why? Why isn't it fresh lik